In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

In [2]:
def read_independed_data(file_name, fill_na_method = 'pad'):
    dataset = pd.read_csv(file_name, parse_dates=["date_recorded"])
    dataset["date_recorded"] = dataset["date_recorded"].map(dt.datetime.toordinal)
    dataset = dataset.fillna(method = fill_na_method)
    return dataset

X = read_independed_data(file_name='X_data.csv', fill_na_method='pad').iloc[:, 1:].values
X_fin_raw = read_independed_data(file_name='test_set.csv', fill_na_method='bfill')

y_dataset = pd.read_csv('y_data.csv')
y = y_dataset.iloc[:, -1].values
X_fin = X_fin_raw.iloc[:, 1:].values

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
x_label_encoder = LabelEncoder()

column_to_encode = [2, 4, 7, 9, 10, 11, 14, 15, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
for i in column_to_encode:
    X[:, i] = x_label_encoder.fit_transform(X[:, i])
    X_fin[:, i] = x_label_encoder.fit_transform(X_fin[:, i])

y_label_encoder = LabelEncoder()
y = y_label_encoder.fit_transform(y)

In [5]:
from sklearn.preprocessing import OneHotEncoder
X_onehotencoder = OneHotEncoder(categorical_features = column_to_encode)
X = X_onehotencoder.fit_transform(X).toarray()
X_fin = X_onehotencoder.transform(X_fin).toarray()

/Users/v.tisov/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/v.tisov/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
X_to_train = X[:]
y_to_train = y[:]

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_to_train = sc.fit_transform(X_to_train)

In [8]:
# backward elimination не получилось сделать (не разобрался до конца) 


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_to_train, y_to_train, test_size = 0.25, random_state = 0)

In [10]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_predict = classifier.predict(X_test)

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
cm

array([[7207,  197,  618],
       [ 613,  341,  157],
       [1394,   87, 4236]])

In [12]:
X_fin = sc.transform(X_fin)
# X_fin = lda.transform(X_fin) # overfitting
y_fin = classifier.predict(X_fin)

In [13]:
result = pd.concat([X_fin_raw.iloc[:,0], pd.Series(y_label_encoder.inverse_transform(y_fin))], axis=1)
result.to_csv("result.csv", index=False, header=["id","status_group"])